**DATA CLEANING AND TIDYING CODE**

In [2]:
import pandas as pd
import numpy as np

# 1. LOADING DATA
df = pd.read_csv('bank_marketing.csv', skipinitialspace=True)

# 2. COLUMN RENAMING & INDEXING
# First, standardize column names (lowercase and underscores)
df.columns = df.columns.str.strip().str.lower().str.replace(r'[()\s%]+', '_', regex=True).str.strip('_')

# Manual mapping for specific columns
rename_map = {
    'year': 'Year', 'age': 'Age', 'month': 'Month', 'job_title': 'Job_Title',
    'region': 'Region', 'educatoin': 'Education', 'maritalstatus': 'Marital_Status',
    'avg monthly income_eur': 'Avg_Monthly_Income_Eur', 'balanceeur': 'Balance_Eur',
    'inflation': 'Inflation_Pct', 'subscribed_termdeposit': 'Subscribed', 'callduration_sec': 'Call_Duration_Sec',
    'contacts_thiscampaign': 'Contacts_This_Campaign', 'creditscore': 'Credit_Score',
    'contacttype': 'Contact_Type', 'customer_id': 'Customer_ID'
}

for col in df.columns:
    if 'interest' in col:
        rename_map[col] = 'Interest_Rate_Pct'

df.rename(columns=rename_map, inplace=True)

if 'Customer_ID' in df.columns:
    df = df.set_index('Customer_ID')

# 3. INITIAL CLEANING & TEXT STANDARDIZATION (CLEANING PART)
# Clean percentage signs and convert to numeric for economic indicators
for col in ['Inflation_Pct', 'Interest_Rate_Pct']:
    if col in df.columns and df[col].dtype == 'object':
        df[col] = df[col].str.replace('%', '').astype(float)

# Global Text Cleaning (Symbols !, #, * and fixing typos)
spelling_fixes = {
    'Centraal': 'Central',
    'Secndary': 'Secondary',
    'Nan': np.nan, 'Na': np.nan, 'None': np.nan
}

cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col] = df[col].astype(str).str.replace(r'[!#*]', '', regex=True)
    df[col] = df[col].str.strip().str.title()
    df[col] = df[col].replace(spelling_fixes)

df.drop_duplicates(inplace=True)
df.dropna(subset=['Subscribed'], inplace=True)
df['Subscribed'] = df['Subscribed'].astype(int)

# 4. HANDLING MISSING VALUES (FILLING PART)
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        if not df[col].mode().empty:
            df[col] = df[col].fillna(df[col].mode()[0])

num_cols = df.select_dtypes(include=['number']).columns
for col in num_cols:
    if col != 'Subscribed':
        df[col] = df[col].fillna(df[col].mean())

# 5. DATE COLUMN CREATION
month_map = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}
df['month_num'] = df['Month'].str[:3].str.title().map(month_map)
df['Year'] = df['Year'].astype(int)

df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['month_num'] + '-01', errors='coerce')
if df['Date'].isnull().any():
    df['Date'] = df['Date'].fillna(df['Date'].mode()[0])

# 6. OUTLIER TREATMENT
outlier_cols = [c for c in num_cols if c not in ['Subscribed', 'Year']]
for col in outlier_cols:
    if col in df.columns:
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        df.loc[(df[col] < lower) | (df[col] > upper), col] = df[col].mean()

# 7. FINAL FORMATTING
# Convert all numerical columns to float (Age, Credit_Score, etc.)
num_cols = df.select_dtypes(include=['number']).columns
df[num_cols] = df[num_cols].astype(float)

# Round everything to 2 decimal places for a clean look
df[num_cols] = df[num_cols].round(2)

# Convert all categorical columns to object
cat_cols = df.select_dtypes(include=['object', 'string']).columns
for col in cat_cols:
    df[col] = df[col].astype('object')

# 8. DROP UNNECESSARY COLUMNS & EXPORT
cols_to_drop = ['month_num', 'Year', 'Month']
df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)

df.to_csv('cleaned_marketing_data.csv', index=True)

print(" FINAL DATA INFO ")
print(df.info())
print("\n FIRST 5 ROWS ")
print(df.head())
print(df.describe())


 FINAL DATA INFO 
<class 'pandas.core.frame.DataFrame'>
Index: 1682 entries, C2100001 to C2001700
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Age                     1682 non-null   float64       
 1   Job_Title               1682 non-null   object        
 2   Marital_Status          1682 non-null   object        
 3   Education               1682 non-null   object        
 4   Region                  1682 non-null   object        
 5   Contact_Type            1682 non-null   object        
 6   avg_monthly_income_eur  1682 non-null   float64       
 7   Balance_Eur             1682 non-null   float64       
 8   Credit_Score            1682 non-null   float64       
 9   Interest_Rate_Pct       1682 non-null   float64       
 10  Inflation_Pct           1682 non-null   float64       
 11  Call_Duration_Sec       1682 non-null   float64       
 12  Contacts_This_Campaign  